In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

In [ ]:
import warnings
from torchinfo import summary

warnings.filterwarnings(action="ignore", category=UserWarning, module="tsdm")

# Load 𝗧ime 𝗦eries 𝗗atasets & 𝗠odels (𝚝𝚜𝚍𝚖)

In [ ]:
import tsdm

## Time Series Datasets

### A **time series** is a tuple $D = (𝐓𝐒, M)$

- Time-indexed data $𝐓𝐒=\{(tᵢ, vᵢ)∣i=1…n\}$
  - timestamps $t∈𝓣$, values $v∈𝓥$
- Time-independent metadata $M∈𝓜$


### A **time series collection** is a tuple $C = (I,S,G)$ consisting of

- Index $I⊆𝓘$ (of id's)
- Collection of timeseries $\{Dᵢ=(𝐓𝐒ᵢ, Mᵢ) ∣ i∈𝓘，𝐓𝐒ᵢ∈(𝓣×𝓥)ᵢ^*， Mᵢ∈𝓜ᵢ\}$
- Index-independent Global data $G∈𝓖$
- If $𝓣ᵢ=𝓣$, $𝓥ᵢ=𝓥$ and $𝓜ᵢ=𝓜$ ⟶ **equimodal**

## Examples

1. Clinical data:
    - index $𝓘$: patient / admission id
    - metadata $M_i$: patient metadata (age, sex, preconditions, ...)
    - values $V_i$: measured data (heart rate, blood pressure, etc.) 
    - globals $G$: units of measurement, measurement devices used, etc.
    
2. Bioreactor data
    - index $𝓘$: experiment / run id
    - metadata $M_i$: bacterial stem used, reactor size, reactor type
    - values $V_i$: measured data (O₂-, Glucose-, Acetate- concentration, stirring speed) 
    - globals $G$: units of measurement, measurement devices used, etc. 

# Load the dataset

In [ ]:
from tsdm.datasets import KIWI_RUNS

dataset = KIWI_RUNS()

In [ ]:
ts = dataset.timeseries

In [ ]:
md = dataset.metadata

In [ ]:
dataset.units

# Load the model & encoder

Preliminary API, ideally should be replaced with a database lookup.

⇝ Encoder depends both on model and dataset.

tags: model, dataset, fold, epoch, hyperparameters

In [ ]:
import tsdm

In [ ]:
from tsdm.models.pretrained import LinODEnet
from torchinfo import summary

model = LinODEnet()
summary(model)

## Make a prediction

In [ ]:
from tsdm.encoders import (
    FrameEncoder,
    MinMaxScaler,
    Standardizer,
    FloatEncoder,
    TensorEncoder,
    Time2Float,
    TimeDeltaEncoder,
)

In [ ]:
s = ts.loc[key]

In [ ]:
self.preprocessor = ChainedEncoder(
    TensorEncoder(),
    DataFrameEncoder(
        Standardizer() @ FloatEncoder(),
        index_encoders=MinMaxScaler() @ DateTimeEncoder(),
    ),
)

In [ ]:
encoder = FrameEncoder(
    Standardizer() @ FloatEncoder(), index_encoders=MinMaxScaler() @ TimeDeltaEncoder()
)

encoder.fit(s)

encoder.encode(s)

In [ ]:
key = (run_id, experiment_id) = (510, 16871)

T, X = ts.loc[key].index, ts.loc[key].astype(float).values

In [ ]:
X

In [ ]:
from zipfile import ZipFile
import pickle

path = "/home/rscholz/.tsdm/models/LinODEnet/linodenet.zip"
with ZipFile(path) as archive:
    with archive.open("encoder.pickle") as file:
        encoder = pickle.load(file)

In [ ]:
from tsdm.models.pretrained import LinODEnetEncoder


encoder = LinODEnetEncoder()

In [ ]:
encoder = LinODEnetEncoder()

# Make a Prediction with the model

# What can models do?

In [ ]:
Level 1: Simple forecasting model
Level 2: Probabilistic forecasting model
Level 3: Forecast across datasets
Level 4: 

## What would be nice to have?

- Data Collection
    - Delayed measurements should come with 2 timestamps. (When was sample collected, when was value recorded)
- Data Validation
    - Categories instead of strings
    - Nullable datatypes for missing values (avoid using e.g. -999 as representation for missing value) 
    - Timedeltas and Datetimes
- Data Annotation - Adding Expert Knowledge
    - Biologically plausible minimal / maximal values possible
- Model database

# Types of temporal data

In [ ]:
%%latex
\begin{table}[H]
\centering
\begin{tabular}{lccc}
   data       & obs.-horizon & pred.-horizon & predicted %
\\ \midrule
   $X$        & {\color{DarkGreen} ✔} & {\color{Crimson} ✘}   & {\color{Crimson} ✘}    %
\\ $U$        & {\color{DarkGreen} ✔} & {\color{DarkGreen} ✔} & {\color{Crimson} ✘}    %
\\ $Y^{auto}$ & {\color{DarkGreen} ✔} & {\color{Crimson} ✘}   & {\color{DarkGreen} ✔}  %
\\ $Y^{pred}$ & {\color{Crimson} ✘}   & {\color{Crimson} ✘}   & {\color{DarkGreen} ✔}  %
\end{tabular}
\end{table}